<a href="https://colab.research.google.com/github/BindiyaB/DataSciencePractice/blob/master/Imputation_Split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from matplotlib import pyplot as plt
from fancyimpute import KNN, NuclearNormMinimization, SoftImpute, BiScaler

In [0]:
from sklearn.datasets import load_diabetes

In [0]:
diabetes=load_diabetes()

In [23]:
diabetes.DESCR

'.. _diabetes_dataset:\n\nDiabetes dataset\n----------------\n\nTen baseline variables, age, sex, body mass index, average blood\npressure, and six blood serum measurements were obtained for each of n =\n442 diabetes patients, as well as the response of interest, a\nquantitative measure of disease progression one year after baseline.\n\n**Data Set Characteristics:**\n\n  :Number of Instances: 442\n\n  :Number of Attributes: First 10 columns are numeric predictive values\n\n  :Target: Column 11 is a quantitative measure of disease progression one year after baseline\n\n  :Attribute Information:\n      - Age\n      - Sex\n      - Body mass index\n      - Average blood pressure\n      - S1\n      - S2\n      - S3\n      - S4\n      - S5\n      - S6\n\nNote: Each of these 10 feature variables have been mean centered and scaled by the standard deviation times `n_samples` (i.e. the sum of squares of each column totals 1).\n\nSource URL:\nhttps://www4.stat.ncsu.edu/~boos/var.select/diabetes.h

In [0]:
X=diabetes.data
Y=diabetes.target

In [25]:
X.shape, Y.shape

((442, 10), (442,))

In [0]:
X=X.copy() # Why do this?????????????

In [0]:
X.ravel()[np.random.choice(X.size, 30, replace=False)] = np.nan # Randomly make 30 values as missing

Imputation Technique

In [45]:
#KN6
X_filled_knn = KNN(k=5).fit_transform(X)

Imputing row 1/442 with 0 missing, elapsed time: 0.042
Imputing row 101/442 with 0 missing, elapsed time: 0.043
Imputing row 201/442 with 0 missing, elapsed time: 0.043
Imputing row 301/442 with 0 missing, elapsed time: 0.043
Imputing row 401/442 with 0 missing, elapsed time: 0.044


In [0]:
#Error
missing_mask = np.isnan(X)

In [46]:
knn_mse = ((X_filled_knn[missing_mask] - diabetes.data[missing_mask]) ** 2).mean()
print("knnImpute MSE: %f" % knn_mse)

#With k=3 --0.001336
#With k=5 -- 0.001111
#With k=10 -- 0.001134
#With k=7, -- 0.001129

#So, keep k =5


knnImpute MSE: 0.001111


In [34]:
X

array([[ 0.03807591,  0.05068012,  0.06169621, ..., -0.00259226,
         0.01990842, -0.01764613],
       [-0.00188202, -0.04464164, -0.05147406, ..., -0.03949338,
        -0.06832974, -0.09220405],
       [ 0.08529891,  0.05068012,  0.04445121, ..., -0.00259226,
         0.00286377, -0.02593034],
       ...,
       [ 0.04170844,  0.05068012, -0.01590626, ..., -0.01107952,
        -0.04687948,  0.01549073],
       [-0.04547248, -0.04464164,  0.03906215, ...,  0.02655962,
         0.04452837, -0.02593034],
       [-0.04547248, -0.04464164, -0.0730303 , ..., -0.03949338,
        -0.00421986,  0.00306441]])